In [1]:
import numpy as np
from src.river import River

env = River()

In [2]:
def policy_iteration(gamma=.9):
    v = np.zeros(env.S.size)
    H = np.zeros(env.S.size)

    F = [env.S0]
    I = []
    G = [env.S0]
    GV = [env.S0]

    end = False

    bell = lambda s:  max([sum([env.T(s,a,s_)*(env.R(s,a,s_) + gamma*v[s_]) for s_ in env.S]) for a in env.A])
      
    greedy = lambda s: np.argmax([sum([env.T(s,a,s_)*(env.R(s,a,s_) + .9*v[s_]) for s_ in env.S]) for a in env.A])
    transit = lambda s, s_: (env.T(s,greedy(s),s_)>0 and s!=s_)
    is_reachable = lambda s_: [s for s in env.S if transit(s,s_ )]
    reachables = lambda s_: {n for s in is_reachable(s_) for n in [s]+list(reachables(s))}

    while not end:
        s = (F+G).pop()
        F.remove(s)
        I += [s]
        F += [x for x in env.S if x not in I and any([env.T(s,a,x)>0 for a in env.A])]
        G = I + F
        Z = [s] + list(reachables(s))
        v = [(bell(s_), H[s_])[s_ in F] if s_ in Z else v[s_] for s_ in env.S]
        GV = [s_ for s_ in G]

        print(v)
        end = len([s for s in env.S if s in list(F+GV) and s not in env.G]) <= 0
    return np.argmax([sum([env.T(s,a,s_)*(env.R(s,a,s_) + .9*v[s_]) for s_ in env.S]) for a in env.A])

        

pi = policy_iteration(gamma=.9)

[-1.0, 0.0, -1.0, -0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[-1.0, 0.0, -1.45, -0.95, 0.0, -1.0, 0.0, 0.0, 0.0, 0.0]


In [34]:
env.plot(pi, True)

 _____________________________ 
|  →  |  →  |  →  |  →  |  ↓  |
|_____|_____|_____|_____|_____|
|  ↑  |  ↑  |  ↑  |  →  |  ↑  |
|_____|_____|_____|_____|_____|

